### Import the libraries

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV

### Load the dataset

In [2]:
df = pd.read_csv(r'data/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


* Sex - Categorical (Nominal) Column - OneHotEncoder
* Age - Numerical Column - Has missing values - SimpleImputer
* Embarked - Categorical (Nominal) Column - Has missing values - SimpleImputer, OneHotEncoder

### Train Test Split

In [6]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [9]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [10]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,male,NaN,1,1,15.2458,C
439,2,male,31.0,0,0,10.5000,S
840,3,male,20.0,0,0,7.9250,S
720,2,female,6.0,0,1,33.0000,S
39,3,female,14.0,1,0,11.2417,C


### ColumnTransformer

In [11]:
preprocessor = ColumnTransformer(
    transformers=[
        ('Age_pipeline', Pipeline(steps = [('imputer', SimpleImputer(strategy='mean')), ('scaler', MinMaxScaler())]), ['Age']),
        ('Fare_pipeline', Pipeline([('scaler', MinMaxScaler())]), ['Fare']), 
        ('Embarked_pipeline', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),('encoder', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore', dtype=np.int32))]), ['Embarked']),
        ('Sex_encoder', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore', dtype=np.int32), ['Sex'])
    ],
    remainder='drop',
    verbose_feature_names_out=True,
    verbose=True
)

### Pipeline

In [12]:
pipe = Pipeline(
    steps=[
        ('Preprocessing', preprocessor),
        ('Classifier', DecisionTreeClassifier(random_state=42))
    ],
    verbose=True
)

#### Fit the model

In [13]:
pipe.fit(X_train, y_train)

[ColumnTransformer] .. (1 of 4) Processing Age_pipeline, total=   0.0s
[ColumnTransformer] . (2 of 4) Processing Fare_pipeline, total=   0.0s
[ColumnTransformer]  (3 of 4) Processing Embarked_pipeline, total=   0.0s
[ColumnTransformer] ... (4 of 4) Processing Sex_encoder, total=   0.0s
[Pipeline] ..... (step 1 of 2) Processing Preprocessing, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing Classifier, total=   0.0s


Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('Age_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age']),
                                                 ('Fare_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Fare']),
                                                 ('Embarked_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.int32'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Embarked']),
                                                 ('Sex_encoder',
                                                  OneHotEncoder(drop='first',
                                                                dtype=<class 'numpy.int32'>,
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Sex'])],
                                   verbose=True)),
                ('Classifier', DecisionTreeClassifier(random_state=42))],
         verbose=True)

### Exploring the Pipeline

In [14]:
pipe.named_steps

{'Preprocessing': ColumnTransformer(transformers=[('Age_pipeline',
                                  Pipeline(steps=[('imputer', SimpleImputer()),
                                                  ('scaler', MinMaxScaler())]),
                                  ['Age']),
                                 ('Fare_pipeline',
                                  Pipeline(steps=[('scaler', MinMaxScaler())]),
                                  ['Fare']),
                                 ('Embarked_pipeline',
                                  Pipeline(steps=[('imputer',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('encoder',
                                                   OneHotEncoder(drop='first',
                                                                 dtype=<class 'numpy.int32'>,
                                                                 handle_unknown='ignore',
                  

In [15]:
pipe.named_steps['Preprocessing'].transformers_[0][1]

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler())])

In [16]:
pipe.named_steps['Preprocessing'].transformers_[0][1][0].statistics_

array([29.49884615])

In [17]:
pipe.feature_names_in_

array(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'],
      dtype=object)

In [18]:
pipe.n_features_in_

7

### Predict

In [19]:
y_pred = pipe.predict(X_test)

In [20]:
accuracy_score(y_test,y_pred)

0.7430167597765364

### Cross Validation Score

In [21]:
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

[ColumnTransformer] .. (1 of 4) Processing Age_pipeline, total=   0.0s
[ColumnTransformer] . (2 of 4) Processing Fare_pipeline, total=   0.0s
[ColumnTransformer]  (3 of 4) Processing Embarked_pipeline, total=   0.0s
[ColumnTransformer] ... (4 of 4) Processing Sex_encoder, total=   0.0s
[Pipeline] ..... (step 1 of 2) Processing Preprocessing, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing Classifier, total=   0.0s
[ColumnTransformer] .. (1 of 4) Processing Age_pipeline, total=   0.0s
[ColumnTransformer] . (2 of 4) Processing Fare_pipeline, total=   0.0s
[ColumnTransformer]  (3 of 4) Processing Embarked_pipeline, total=   0.0s
[ColumnTransformer] ... (4 of 4) Processing Sex_encoder, total=   0.0s
[Pipeline] ..... (step 1 of 2) Processing Preprocessing, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing Classifier, total=   0.0s
[ColumnTransformer] .. (1 of 4) Processing Age_pipeline, total=   0.0s
[ColumnTransformer] . (2 of 4) Processing Fare_pipeline, total=   0.0s


0.7472963656062247

### Hyperparamter Tuning (GridSearch CV)

In [22]:
params = {
    'Classifier__max_depth': [1, 2, 3, 4, 5, None]
}

grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

[ColumnTransformer] .. (1 of 4) Processing Age_pipeline, total=   0.0s
[ColumnTransformer] . (2 of 4) Processing Fare_pipeline, total=   0.0s
[ColumnTransformer]  (3 of 4) Processing Embarked_pipeline, total=   0.0s
[ColumnTransformer] ... (4 of 4) Processing Sex_encoder, total=   0.0s
[Pipeline] ..... (step 1 of 2) Processing Preprocessing, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing Classifier, total=   0.0s
[ColumnTransformer] .. (1 of 4) Processing Age_pipeline, total=   0.0s
[ColumnTransformer] . (2 of 4) Processing Fare_pipeline, total=   0.0s
[ColumnTransformer]  (3 of 4) Processing Embarked_pipeline, total=   0.0s
[ColumnTransformer] ... (4 of 4) Processing Sex_encoder, total=   0.0s
[Pipeline] ..... (step 1 of 2) Processing Preprocessing, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing Classifier, total=   0.0s
[ColumnTransformer] .. (1 of 4) Processing Age_pipeline, total=   0.0s
[ColumnTransformer] . (2 of 4) Processing Fare_pipeline, total=   0.0s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Preprocessing',
                                        ColumnTransformer(transformers=[('Age_pipeline',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['Age']),
                                                                        ('Fare_pipeline',
                                                                         Pipeline(steps=[('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['Fare']),
                                                                        ('Embarked_pipeline',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),...
                                                                                                        dtype=<class 'numpy.int32'>,
                                                                                                        handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['Embarked']),
                                                                        ('Sex_encoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       dtype=<class 'numpy.int32'>,
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['Sex'])],
                                                          verbose=True)),
                                       ('Classifier',
                                        DecisionTreeClassifier(random_state=42))],
                                verbose=True),
             param_grid={'Classifier__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [23]:
grid.best_score_

0.794927607603664

In [24]:
grid.best_params_

{'Classifier__max_depth': 4}

### Exporting the Pipeline

In [25]:
pickle.dump(pipe, open('pipe.pkl', 'wb'))

### Predict using Test Data

In [26]:
pipe_pkl = pickle.load(open('pipe.pkl', 'rb'))

In [27]:
test_input = pd.DataFrame([[
    2,         # Pclass
    'male',    # Sex
    24.0,      # Age
    0,         # SibSp
    0,         # Parch
    25.5,      # Fare
    'S'        # Embarked
]], columns=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])

In [28]:
test_input

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,2,male,24.0,0,0,25.5,S


In [29]:
pipe.predict(test_input)

array([0], dtype=int64)

0 -> Does not Survive

Check out this blog post - https://medium.com/@sahin.samia/scikit-learn-pipelines-explained-streamline-and-optimize-your-machine-learning-processes-f17b1beb86a4